Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.1 MB/s eta 0:00:00


1. Cirq ile Dolanıklık Takası

In [2]:
import cirq
import numpy as np

# Qubit tanımları
A, C, Cp, B = cirq.LineQubit.range(4)

# Devre
circuit = cirq.Circuit()

# 1️⃣ Ayşe-Caner ve Caner'-Burak dolanıklığı
circuit.append([cirq.H(A), cirq.CNOT(A, C)])
circuit.append([cirq.H(Cp), cirq.CNOT(Cp, B)])

# 2️⃣ Caner'in (C, C') üzerinde Bell ölçümü
circuit.append([cirq.CNOT(C, Cp), cirq.H(C)])
circuit.append([cirq.measure(C, Cp, key='bell_meas')])

# Simülasyon
sim = cirq.Simulator()
result = sim.run(circuit, repetitions=1)
measurement = result.measurements['bell_meas'][0]
print("Caner'in ölçüm sonucu:", measurement)

# 3️⃣ Ölçüm sonucuna göre düzeltme işlemleri
corrections = {
    (0, 0): [],
    (0, 1): [cirq.Z(B)],
    (1, 0): [cirq.X(B)],
    (1, 1): [cirq.Z(B), cirq.X(B)]
}

circuit.append(corrections[tuple(measurement)])

# Son devre
print("\nDolanıklık Takası Devresi:\n")
print(circuit)

# Durum vektörü simülasyonu
state = sim.simulate(circuit).final_state_vector
print("\nSon Kuantum Durumu (|ψ⟩):")
print(np.round(state, 3))


Caner'in ölçüm sonucu: [1 0]

Dolanıklık Takası Devresi:

0: ───H───@────────────────────────────
          │
1: ───────X───@───H───M('bell_meas')───
              │       │
2: ───H───@───X───────M────────────────
          │
3: ───────X───X────────────────────────

Son Kuantum Durumu (|ψ⟩):
[0.   +0.j 0.   +0.j 0.707+0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j
 0.   +0.j 0.   +0.j 0.   +0.j 0.   +0.j 0.707+0.j 0.   +0.j 0.   +0.j
 0.   +0.j 0.   +0.j]


2. Qiskit ile Dolanıklık Takası

In [6]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_state_city

# 4 qubit (A, C, C', B)
qc = QuantumCircuit(4, 2)

# 1️⃣ Ayşe-Caner ve Caner'-Burak dolanıklığı
qc.h(0)
qc.cx(0, 1)
qc.h(2)
qc.cx(2, 3)

# 2️⃣ Caner (1,2) üzerinde Bell ölçümü
qc.cx(1, 2)
qc.h(1)
qc.measure([1, 2], [0, 1])

# 3️⃣ Klasik ölçüm sonuçlarına göre düzeltme
qc.x(3).c_if(qc.cregs[0], 2)  # 10 durumu -> X
qc.z(3).c_if(qc.cregs[0], 1)  # 01 durumu -> Z
qc.x(3).c_if(qc.cregs[0], 3)  # 11 durumu -> ZX

# Görselleştir
print(qc.draw())

# Simülasyon
sim = Aer.get_backend('statevector_simulator')
result = execute(qc.remove_final_measurements(inplace=False), sim).result()
state = result.get_statevector()
print("\nSon durum vektörü:\n", state)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

3. QuTiP ile Dolanıklık Takası

In [5]:
from qutip import *
import numpy as np

# Bell durum fonksiyonu
def bell_state():
    return (tensor(basis(2,0), basis(2,0)) + tensor(basis(2,1), basis(2,1))).unit()

# Başlangıç durumları: |Φ+>_AC ⊗ |Φ+>_C'B
phi_AC = bell_state()
phi_CpB = bell_state()
psi_init = tensor(phi_AC, phi_CpB)

# Bell ölçüm operatörleri
bell_basis = [
    (tensor(basis(2,0), basis(2,0)) + tensor(basis(2,1), basis(2,1))).unit(),  # Φ+
    (tensor(basis(2,0), basis(2,0)) - tensor(basis(2,1), basis(2,1))).unit(),  # Φ-
    (tensor(basis(2,0), basis(2,1)) + tensor(basis(2,1), basis(2,0))).unit(),  # Ψ+
    (tensor(basis(2,0), basis(2,1)) - tensor(basis(2,1), basis(2,0))).unit(),  # Ψ-
]

# Ölçüm: C ve C' qubitleri (2 ve 3)
rho = ket2dm(psi_init)
proj_ops = [tensor(qeye(2), ket2dm(bell), qeye(2)) for bell in bell_basis]

# Ölçüm olasılıkları
probs = [expect(P, rho) for P in proj_ops]
print("Ölçüm olasılıkları:", np.round(probs, 3))

# Ölçüm sonucu Φ+ durumu olsun (en olası)
rho_post = proj_ops[0] * rho * proj_ops[0]
rho_post = rho_post / rho_post.tr()

# Ayşe-Burak (0 ve 3) indirgenmiş yoğunluk matrisi
rho_AB = rho_post.ptrace([0, 3])
print("\nAyşe-Burak yoğunluk matrisi:\n", rho_AB)
print("Fidelity(|Φ+>):", fidelity(rho_AB, bell_state()))


Ölçüm olasılıkları: [0.25 0.25 0.25 0.25]

Ayşe-Burak yoğunluk matrisi:
 Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
Fidelity(|Φ+>): 0.9999999999999998
